# import libraries and setting instaloader

In [1]:
import instaloader
import datetime
import warnings
import time
warnings.filterwarnings("ignore")
import time
from instaloader import Instaloader,Profile
L=Instaloader()

# define API functions

In [2]:
#username to profile#
def anyname2user(anyname):
    if type(anyname)==int:
        profile=Profile.from_id(L.context,anyname)
    else:
        profile=Profile.from_username(L.context,anyname)
    return profile

In [3]:
#username to userid#
def user2accountid(user):
    userid=user.userid
    return userid

In [4]:
#userid to personname#
def user2name(user):
    personname=user.full_name
    return personname
    

In [5]:
#userid to username#
def user2username(user):
    username=user.username
    return username

In [6]:
#userid to followerscount#
def user2followerscount(user):
    followerscount=user.followers
    return followerscount

In [7]:
#userid to postscount#
def user2postscount(user):
    postscount=user.mediacount
    return postscount

In [8]:
#post to postid#
def post2postid(post):
    postid=post.mediaid
    return postid

In [9]:
#post to snsuserid#
def post2snsuserid(post):
    snsuserid=post.owner_id
    return snsuserid

In [10]:
#post to content#
def post2content(post):
    if len(list(post.get_comments())) ==0:
        content=" "
    else:
        content=list(post.get_comments())[0].text
    return content

In [11]:
#post to favorite count#
def post2favoritecount(post):
    favoritecount=post.likes
    return favoritecount

In [12]:
#post to createdtime#
def post2createdtime(post):
    createdtime=post.date_utc
    return createdtime

In [13]:
#user to locationname#
def user2locationname(user):
    postslist=list(user.get_posts())
    Locationname = " "
    x=0
    timenow=datetime.datetime.now()
    start=time.clock()
    for post in postslist:
        x=x+1
        timeused=time.clock()-start
        if timeused>5:
            Locationname = " "
            break
        if hasattr(post, 'location'):
            if hasattr(post.location, 'name'):
                locationname=post.location.name
                break
        if x==5:
            Locationname = " "
            break
    return Locationname

In [14]:
#post to postlocation#
def post2postlocation(post):
    if hasattr(post, 'location'):
        if hasattr(post.location, 'name'):
            postlocation=post.location.name
    else:
        postlocation=" "
    return postlocation

In [15]:
#userid to posts#
def useridtoposts(userid):
    posts=anyname2user(userid).get_posts()
    return posts

In [16]:
#userid to 30posts#
def anyname2dayspostlist(username, number):
    userid=Profile.from_username(L.context,username).userid
    postlist=[]
    timenow=datetime.datetime.now()
    posts=anyname2user(userid).get_posts()
    for post in posts:
        if(timenow-post2createdtime(post)).days<number:
            postlist.append(post)
        else:
            break
    return postlist

In [17]:
#hashtag to 30posts#
def tagtexthashtag2postlist(hashtag,number):
    x=0
    postlist=[]
    posts=L.get_hashtag_posts(hashtag)
    for post in posts:
        postlist.append(post)
        x=x+1
        if x==number:
            break
    return postlist

In [18]:
#post to hashtags#
def post2tagtexthashtaglist(post):
    hashtaglist=[]
    hashtags=post.caption_hashtags
    for hashtag in hashtags:
        hashtaglist.append(hashtag)
    return hashtags

In [19]:
#post to tagtextmentionlist#
def post2tagtextmentionlist(post):
    mentionlist=[]
    mentions=post.caption_mentions
    for mention in mentions:
        mentionlist.append(mention)
    return mentions

# Make DataFrame

In [20]:
import pandas as pd
Player = pd.DataFrame(columns=["personID"])
Player_Fan = pd.DataFrame(columns=["prfID", "PlayerpersonID", "FanpersonID"])
Fan = pd.DataFrame(columns=["personID"])
Fan_Team = pd.DataFrame(columns=["frtID", "FanpersonID", "TeamteamID"])
Person = pd.DataFrame(columns=["personID", "personName", "locationID"])
Team = pd.DataFrame(columns=["teamID", "teamName", "locationID"])
Person_SNSUser = pd.DataFrame(columns=["prsID", "PersonpersonID", "SNSUserSNSUserID"])
Location = pd.DataFrame(columns=["locationID", "locationName"])
Team_SNSUser = pd.DataFrame(columns=["trsID", "TeamteamID", "SNSUserSNSUserID"])
SNSUser = pd.DataFrame(columns=["SNSUserID", "SNSType", "accountID", "userName", "followersCount", "postsCount"])
Post = pd.DataFrame(columns=["SNSpostID", "postID", "SNSUserID", "content", "favoriteCount", "createdTime"])
Post_Tag = pd.DataFrame(columns=["prtID", "PostSNSpostID", "TagTagID"])
Tag = pd.DataFrame(columns=["tagID", "tagText", "tagType"])
DomainTag = pd.DataFrame(columns=["tagID"])
SynonymsTag = pd.DataFrame(columns=["tagID1", "tagID2"])
MisspellingsTag = pd.DataFrame(columns=["tagID1", "tagID2"])
TagCatagory_Tag = pd.DataFrame(columns=["ttrID", "TagCatagorycatagoryID", "TagTagID"])
TagCatagory = pd.DataFrame(columns=["catagoryID", "catagoryName"])

# Data Retrieving Function

In [21]:
# user saving function
def savesnsuser(user):
    if SNSUser[SNSUser["accountID"].isin([user2accountid(user)])].shape[0] == 0:
        SNSUser.loc[SNSUser.shape[0] + 1] = {"SNSUserID": SNSUser.shape[0] + 1, 
                                             "SNSType": "Twitter", 
                                             "accountID": user2accountid(user), 
                                             "userName": user2username(user), 
                                             "followersCount": user2followerscount(user), 
                                             "postsCount":user2postscount(user)}
    else:
        SNSUser.set_value(SNSUser[SNSUser["accountID"].isin([user2accountid(user)])].loc[:, ["SNSUserID"]].values[0][0], "followersCount", user2followerscount(user))
        SNSUser.set_value(SNSUser[SNSUser["accountID"].isin([user2accountid(user)])].loc[:, ["SNSUserID"]].values[0][0], "postsCount", user2postscount(user))

In [22]:
# post saving function
def savesnspost(post):
    if Post[Post["postID"].isin([post2postid(post)])].shape[0] == 0:
        Post.loc[Post.shape[0] + 1] = {"SNSpostID": Post.shape[0] + 1, 
                                       "postID": post2postid(post), 
                                       "SNSUserID": SNSUser[(SNSUser["accountID"].isin([post2snsuserid(post)])) & (SNSUser["SNSType"] == "Twitter")].loc[:, ["SNSUserID"]].values[0][0], 
                                       "content":post2content(post), 
                                       "favoriteCount":post2favoritecount(post), 
                                       "createdTime":post2createdtime(post)}
        taglist = post2tagtexthashtaglist(post)
        for tag in taglist:
            if Tag[(Tag["tagText"].isin([tag])) & (Tag["tagType"] == "hashtag")].shape[0] == 0:
                Tag.loc[Tag.shape[0] + 1] = {"tagID": Tag.shape[0] + 1, 
                                             "tagText": tag, 
                                             "tagType": "hashtag"}
            Post_Tag.loc[Post_Tag.shape[0] + 1] = {"prtID": Post_Tag.shape[0] + 1, 
                                                   "PostSNSpostID": Post.shape[0], 
                                                   "TagTagID": Tag[(Tag["tagText"].isin([tag])) & (Tag["tagType"] == "hashtag")].loc[:, ["tagID"]].values[0][0]}
        taglist=post2tagtextmentionlist(post)
        for tag in taglist:
            if Tag[(Tag["tagText"].isin([tag])) & (Tag["tagType"] == "mention")].shape[0] == 0:
                Tag.loc[Tag.shape[0] + 1] = {"tagID": Tag.shape[0] + 1, 
                                             "tagText": tag, 
                                             "tagType": "mention"}
            Post_Tag.loc[Post_Tag.shape[0] + 1] = {"prtID": Post_Tag.shape[0] + 1, 
                                                   "PostSNSpostID": Post.shape[0], 
                                                   "TagTagID": Tag[(Tag["tagText"].isin([tag])) & (Tag["tagType"] == "mention")].loc[:, ["tagID"]].values[0][0]}
    else:
        Post.set_value(Post[Post["postID"].isin([post2postid(post)])].loc[:, ["SNSpostID"]].values[0][0], "favoriteCount", post2favoritecount(post))

In [23]:
# team saving function
def saveteam(teamName, userName):
    teamID = 0
    teamIDtemp = 0
    userExistFlag = True
    if SNSUser[SNSUser["accountID"].isin([user2accountid(anyname2user(userName))])].shape[0] == 0:
        userExistFlag = False
    savesnsuser(anyname2user(userName))
    locationName = user2locationname(anyname2user(userName))
    if Location[Location["locationName"].isin([locationName])].shape[0] == 0:
        Location.loc[Location.shape[0] + 1] = {"locationID": Location.shape[0] + 1, 
                                               "locationName": locationName}
    if not userExistFlag:
        Team.loc[Team.shape[0] + 1] = {"teamID": Team.shape[0] + 1, 
                                       "teamName": teamName, 
                                       "locationID": Location[Location["locationName"].isin([locationName])].loc[:, ["locationID"]].values[0][0]}
        teamIDtemp = Team.shape[0]
        teamID = Team.shape[0]
    else:
        teamID = Team[Team["teamName"].isin([teamName])].loc[:, ["teamID"]].values[0][0]
        if (teamIDtemp != 0):
            teamID = teamIDtemp
        Team.set_value(teamID, "locationID", Location[Location["locationName"].isin([locationName])].loc[:, ["locationID"]].values[0][0])
    Team_SNSUser.loc[Team_SNSUser.shape[0] + 1] = {"trsID": Team_SNSUser.shape[0] + 1, 
                                                   "TeamteamID": teamID, 
                                                   "SNSUserSNSUserID": SNSUser[SNSUser["userName"].isin([userName])].loc[:, ["SNSUserID"]].values[0][0]}
    Team_SNSUser.drop_duplicates(subset=["TeamteamID","SNSUserSNSUserID"], keep='first', inplace=False)

In [24]:
# player saving function
def saveplayer(personName, userName):
    personID = 0
    personIDtemp = 0
    userExistFlag = True
    if SNSUser[SNSUser["accountID"].isin([user2accountid(anyname2user(userName))])].shape[0] == 0:
        userExistFlag = False
    savesnsuser(anyname2user(userName))
    locationName = user2locationname(anyname2user(userName))
    if Location[Location["locationName"].isin([locationName])].shape[0] == 0:
        Location.loc[Location.shape[0] + 1] = {"locationID": Location.shape[0] + 1, 
                                               "locationName": locationName}
    if not userExistFlag:
        Person.loc[Person.shape[0] + 1] = {"personID": Person.shape[0] + 1, 
                                           "personName": personName, 
                                           "locationID": Location[Location["locationName"].isin([locationName])].loc[:, ["locationID"]].values[0][0]}
        Player.loc[Player.shape[0] + 1] = {"personID": Person.shape[0]}
        personIDtemp = Person.shape[0]
        personID = Person.shape[0]
    else:
        personID = Person[Person["personName"].isin([personName])].loc[:, ["personID"]].values[0][0]
        if (personIDtemp != 0):
            personID = personIDtemp
        Person.set_value(Person[Person["personName"].isin([personName])].loc[:, ["personID"]].values[0][0], "locationID", Location[Location["locationName"].isin([locationName])].loc[:, ["locationID"]].values[0][0])
    Person_SNSUser.loc[Person_SNSUser.shape[0] + 1] = {"prsID": Person_SNSUser.shape[0] + 1, 
                                                       "PersonpersonID": personID, 
                                                       "SNSUserSNSUserID": SNSUser[SNSUser["userName"].isin([userName])].loc[:, ["SNSUserID"]].values[0][0]}
    Person_SNSUser.drop_duplicates(subset=["PersonpersonID","SNSUserSNSUserID"], keep='first', inplace=False)

In [25]:
# save user's posts
def saveposts(userName, number):
    postlist = anyname2dayspostlist(userName,number)
    for post in postlist:
        savesnspost(post)

In [26]:
# save posts with team #hashtag
def saveteamhashtagposts(hashtag, number):
    postlist = tagtexthashtag2postlist(hashtag, number)
    for post in postlist:
        user = anyname2user(post2snsuserid(post))
        userName = user2username(user)
        personName = user2name(user)
        personID = 0
        userExistFlag = True
        if SNSUser[SNSUser["accountID"].isin([user2accountid(anyname2user(userName))])].shape[0] == 0:
            userExistFlag = False
        personExistFlag = True
        if Person[Person["personName"].isin([personName])].shape[0] == 0: 
            personExistFlag = False
        savesnsuser(anyname2user(userName))
        locationName = user2locationname(anyname2user(userName))
        if Location[Location["locationName"].isin([locationName])].shape[0] == 0:
            Location.loc[Location.shape[0] + 1] = {"locationID": Location.shape[0] + 1, 
                                                   "locationName": locationName}
        if not personExistFlag:
            Person.loc[Person.shape[0] + 1] = {"personID": Person.shape[0] + 1, 
                                               "personName": personName, 
                                               "locationID": Location[Location["locationName"].isin([locationName])].loc[:, ["locationID"]].values[0][0]}
            personID = Person.shape[0]
        else:
            personID = Person[Person["personName"].isin([personName])].loc[:, ["personID"]].values[0][0]
            Person.set_value(Person[Person["personName"].isin([personName])].loc[:, ["personID"]].values[0][0], "locationID", Location[Location["locationName"].isin([locationName])].loc[:, ["locationID"]].values[0][0])
        if (not userExistFlag) | (not personExistFlag):
            Person_SNSUser.loc[Person_SNSUser.shape[0] + 1] = {"prsID": Person_SNSUser.shape[0] + 1, 
                                                               "PersonpersonID": personID, 
                                                               "SNSUserSNSUserID": SNSUser[SNSUser["userName"].isin([userName])].loc[:, ["SNSUserID"]].values[0][0]}
            Person_SNSUser.drop_duplicates(subset=["PersonpersonID","SNSUserSNSUserID"], keep='first', inplace=False)
        if Fan[Fan["personID"].isin([personID])].shape[0] == 0:
            Fan.loc[Fan.shape[0] + 1] = {"personID": personID}
        teamsnsid = SNSUser[SNSUser["userName"].isin([hashtag])].loc[:, ["SNSUserID"]].values[0][0]
        teamteamid = Team_SNSUser[Team_SNSUser["SNSUserSNSUserID"].isin([teamsnsid])].loc[:, ["TeamteamID"]].values[0][0]
        teamName = Team[Team["teamID"].isin([teamteamid])].loc[:, ["teamName"]].values[0][0]
        Fan_Team.loc[Fan_Team.shape[0] + 1] = {"frtID": Fan_Team.shape[0] + 1, 
                                               "FanpersonID": personID, 
                                               "TeamteamID": Team[Team["teamName"].isin([teamName])].loc[:, ["teamID"]].values[0][0]}
        Fan_Team.drop_duplicates(subset=["FanpersonID","TeamteamID"], keep='first', inplace=False)
        savesnspost(post)

In [27]:
# save posts with player #hashtag
def saveplayerhashtagposts(hashtag, number):
    postlist = tagtexthashtag2postlist(hashtag, number)
    for post in postlist:
        user = anyname2user(post2snsuserid(post))
        userName = user2username(user)
        personName = user2name(user)
        personID = 0
        userExistFlag = True
        if SNSUser[SNSUser["accountID"].isin([user2accountid(anyname2user(userName))])].shape[0] == 0:
            userExistFlag = False
        personExistFlag = True
        if Person[Person["personName"].isin([personName])].shape[0] == 0: 
            personExistFlag = False
        savesnsuser(anyname2user(userName))
        locationName = user2locationname(anyname2user(userName))
        if Location[Location["locationName"].isin([locationName])].shape[0] == 0:
            Location.loc[Location.shape[0] + 1] = {"locationID": Location.shape[0] + 1, 
                                                   "locationName": locationName}
        if not personExistFlag:
            Person.loc[Person.shape[0] + 1] = {"personID": Person.shape[0] + 1, 
                                               "personName": personName, 
                                               "locationID": Location[Location["locationName"].isin([locationName])].loc[:, ["locationID"]].values[0][0]}
            personID = Person.shape[0]
        else:
            personID = Person[Person["personName"].isin([personName])].loc[:, ["personID"]].values[0][0]
            Person.set_value(Person[Person["personName"].isin([personName])].loc[:, ["personID"]].values[0][0], "locationID", Location[Location["locationName"].isin([locationName])].loc[:, ["locationID"]].values[0][0])
        if (not userExistFlag) | (not personExistFlag):
            Person_SNSUser.loc[Person_SNSUser.shape[0] + 1] = {"prsID": Person_SNSUser.shape[0] + 1, 
                                                               "PersonpersonID": personID, 
                                                               "SNSUserSNSUserID": SNSUser[SNSUser["userName"].isin([userName])].loc[:, ["SNSUserID"]].values[0][0]}
            Person_SNSUser.drop_duplicates(subset=["PersonpersonID","SNSUserSNSUserID"], keep='first', inplace=False)
        if Fan[Fan["personID"].isin([personID])].shape[0] == 0:
            Fan.loc[Fan.shape[0] + 1] = {"personID": personID}
        playersnsid = SNSUser[SNSUser["userName"].isin([hashtag])].loc[:, ["SNSUserID"]].values[0][0]
        personpersonid = Person_SNSUser[Person_SNSUser["SNSUserSNSUserID"].isin([playersnsid])].loc[:, ["PersonpersonID"]].values[0][0]
        personName = Person[Person["personID"].isin([personpersonid])].loc[:, ["personName"]].values[0][0]
        Player_Fan.loc[Player_Fan.shape[0] + 1] = {"prfID": Player_Fan.shape[0] + 1, 
                                                   "PlayerpersonID": Person[Person["personName"].isin([personName])].loc[:, ["personID"]].values[0][0], 
                                                   "FanpersonID": personID}
        Player_Fan.drop_duplicates(subset=["PlayerpersonID","FanpersonID"], keep='first', inplace=False)
        savesnspost(post)

# Saving

## save team and players

In [28]:
try:
    saveteam("New York Yankees", "yankees")
    print("1")
    saveteam("Los Angeles Dodgers", "dodgers")
    print("2")
    saveteam("Tampa Bay Rays", "raysbaseball")
    print("3")
    saveplayer("Greg Bird", "_gregbird33")
    print("4")
    saveplayer("DJ LeMahieu", "dj_lemahieu")
    print("5")
    saveplayer("Zack Britton", "zbritton")
    print("6")
    saveplayer("Max Muncy", "maxmuncy")
    print("7")
    saveplayer("Julio Urías", "theteenager7")
    print("8")
    saveplayer("Cody Bellinger", "cody_bellinger")
    print("9")
    saveplayer("Christian Arroyo", "arroyoc22")
    print("10")
    saveplayer("Austin Meadows", "austinmeadows13")
    print("11")
    saveplayer("Guillermo Heredia", "heredia54")
    print("12")
except:
    pass

1


JSON Query to graphql/query: 429 Too Many Requests [retrying; skip with ^C]
HTTP error code 429 was returned because too many queries occured in
the last time. Please do not use Instagram in your browser or run
multiple instances of Instaloader in parallel.
GraphQL requests:
  472f257a40c653c64c666ce877d59d2b
    last 10 minutes: 179 requests
    last 11 minutes: 179 requests
    last 15 minutes: 179 requests
    last 20 minutes: 179 requests
    last 30 minutes: 179 requests
    last 60 minutes: 179 requests


The request will be retried in 315 seconds, at 06:23.
2

Too many queries in the last time. Need to wait 307 seconds, until 06:34.
3
4
5
6


JSON Query to maxmuncy/: ('Connection aborted.', OSError("(54, 'ECONNRESET')")) [retrying; skip with ^C]


7
8
9
10
11


JSON Query to explore/locations/3060/: ('Connection aborted.', OSError("(54, 'ECONNRESET')")) [retrying; skip with ^C]


12


## save teams' and players' posts

In [29]:
days = 1
try:
    saveposts("yankees",days)
    print("1")
    saveposts("dodgers",days)
    print("2")
    saveposts("raysbaseball",days)
    print("3")
    saveposts("_gregbird33",days)
    print("4")
    saveposts("dj_lemahieu",days)
    print("5")
    saveposts("zbritton",days)
    print("6")
    saveposts("maxmuncy",days)
    print("7")
    saveposts("theteenager7",days)
    print("8")
    saveposts("cody_bellinger",days)
    print("9")
    saveposts("arroyoc22",days)
    print("10")
    saveposts("austinmeadows13",days)
    print("11")
    saveposts("heredia54",days)
    print("12")
except:
    pass

1
2
3
4
5
6
7
8
9
10
11
12


## save tags posts

In [30]:
postscount = 1
try:
    saveteamhashtagposts("yankees", postscount)
    print("1")
    saveteamhashtagposts("dodgers", postscount)
    print("2")
    saveteamhashtagposts("raysbaseball", postscount)
    print("3")
    saveplayerhashtagposts("_gregbird33", postscount)
    print("4")
    saveplayerhashtagposts("dj_lemahieu", postscount)
    print("5")
    saveplayerhashtagposts("zbritton", postscount)
    print("6")
    saveplayerhashtagposts("maxmuncy", postscount)
    print("7")
    saveplayerhashtagposts("theteenager7", postscount)
    print("8")
    saveplayerhashtagposts("cody_bellinger", postscount)
    print("9")
    saveplayerhashtagposts("arroyoc22", postscount)
    print("10")
    saveplayerhashtagposts("austinmeadows13", postscount)
    print("11")
    saveplayerhashtagposts("heredia54", postscount)
    print("12")
except:
    pass

1
2
3

Too many queries in the last time. Need to wait 163 seconds, until 07:14.

Too many queries in the last time. Need to wait 17 seconds, until 07:15.

Too many queries in the last time. Need to wait 15 seconds, until 07:15.
4


# tables

In [31]:
Player.head()

,personID
1,1
2,2
3,3
4,4
5,5


In [32]:
Player_Fan.head()

,prfID,PlayerpersonID,FanpersonID
1,1,1,13


In [33]:
Fan.head()

,personID
1,10
2,11
3,12
4,13


In [34]:
Fan_Team.head()

,frtID,FanpersonID,TeamteamID
1,1,10,1
2,2,11,2
3,3,12,3


In [35]:
Person.head()

,personID,personName,locationID
1,1,Greg Bird,1
2,2,DJ LeMahieu,1
3,3,Zack Britton,1
4,4,Max Muncy,1
5,5,Julio Urías,1


In [36]:
Team.head()

,teamID,teamName,locationID
1,1,New York Yankees,1
2,2,Los Angeles Dodgers,1
3,3,Tampa Bay Rays,1


In [37]:
Person_SNSUser.head()

,prsID,PersonpersonID,SNSUserSNSUserID
1,1,1,4
2,2,2,5
3,3,3,6
4,4,4,7
5,5,5,8


In [38]:
Location.head()

,locationID,locationName
1,1,


In [39]:
Team_SNSUser.head()

,trsID,TeamteamID,SNSUserSNSUserID
1,1,1,1
2,2,2,2
3,3,3,3


In [40]:
SNSUser.head()

,SNSUserID,SNSType,accountID,userName,followersCount,postsCount
1,1,Twitter,46757590,yankees,2202290,5008
2,2,Twitter,46782831,dodgers,1938260,12181
3,3,Twitter,47288466,raysbaseball,264761,7638
4,4,Twitter,4479471161,_gregbird33,107617,46
5,5,Twitter,10499719502,dj_lemahieu,5733,3


In [41]:
Post.head()

,SNSpostID,postID,SNSUserID,content,favoriteCount,createdTime
1,1,2030199101733336830,2,Can we a do a friends night ?❤️,12119,2019-04-26 00:30:14
2,2,2030165521294879068,2,Amazing pic⚾️🔥,12088,2019-04-25 23:23:31
3,3,2030134237071760378,2,KJ keeps giving up the booty!!!,24924,2019-04-25 22:21:21
4,4,2030121918258333239,2,"Bullpen was solid... they deserve credit, they...",23631,2019-04-25 21:56:53
5,5,2030113054754358984,2,@bodaciousbearr @manintheground lol the comments,40103,2019-04-25 21:39:16


In [42]:
Post_Tag.head()

,prtID,PostSNSpostID,TagTagID
1,1,1,1
2,2,2,2
3,3,3,2
4,4,3,3
5,5,6,4


In [43]:
Tag.head()

,tagID,tagText,tagType
1,1,seinfeldtv,mention
2,2,dodgers,hashtag
3,3,ross_stripling,mention
4,4,kenleyjansen74,mention
5,5,dugie11,mention


In [44]:
DomainTag.head()

,tagID


In [45]:
SynonymsTag.head()

,tagID1,tagID2


In [46]:
MisspellingsTag.head()

,tagID1,tagID2


In [47]:
TagCatagory_Tag.head()

,ttrID,TagCatagorycatagoryID,TagTagID


In [48]:
TagCatagory.head()

,catagoryID,catagoryName


In [49]:
PlayerList = Player.values.tolist()
Player_FanList = Player_Fan.values.tolist()
FanList = Fan.values.tolist()
Fan_TeamList = Fan_Team.values.tolist()
PersonList = Person.values.tolist()
TeamList = Team.values.tolist()
Person_SNSUserList = Person_SNSUser.values.tolist()
LocationList = Location.values.tolist()
Team_SNSUserList = Team_SNSUser.values.tolist()
SNSUserList = SNSUser.values.tolist()
PostList = Post.values.tolist()
Post_TagList = Post_Tag.values.tolist()
TagList = Tag.values.tolist()
DomainTagList = DomainTag.values.tolist()
SynonymsTagList = SynonymsTag.values.tolist()
MisspellingsTagList = MisspellingsTag.values.tolist()
TagCatagory_TagList = TagCatagory_Tag.values.tolist()
TagCatagoryList = TagCatagory.values.tolist()

# Tagging

## domain tag

In [51]:
teamSNS=[["New York Yankees", "yankees"],
         ["Los Angeles Dodgers", "dodgers"],
         ["Tampa Bay Rays", "raysbaseball"]]
playerSNS=[["Greg Bird", "_gregbird33"],
           ["DJ LeMahieu", "dj_lemahieu"],
           ["Zack Britton", "zbritton"],
           ["Max Muncy", "maxmuncy"],
           ["Julio Urías", "theteenager7"],
           ["Cody Bellinger", "cody_bellinger"],
           ["Christian Arroyo", "arroyoc22"],
           ["Austin Meadows", "austinmeadows13"],
           ["Guillermo Heredia", "heredia54"]]

In [52]:
for tag in TagList:
    if tag[2]=="hashtag":
        for player in playerSNS:
            if tag[1] in player[1]:
                DomainTagList.append([tag[0]])
        for team in teamSNS:
            if tag[1] in team[1]:
                DomainTagList.append([tag[0]])

## synonyms tag

In [53]:
def tagcount1(tagid):
    counter = 0
    for pt in Post_TagList:
        if pt[2]==tagid:
            counter=counter+1
    return counter
tagcount1(5)

1

In [54]:
def tagcount2(tagid1, tagid2):
    counter=0
    tagid1list = []
    for posttag in Post_TagList:
        if posttag[2]==tagid1:
            tagid1list.append(posttag[1])
    tagid2list = []
    for posttag in Post_TagList:
        if posttag[2]==tagid2:
            tagid2list.append(posttag[1])
    for tagid1 in tagid1list:
        for tagid2 in tagid2list:
            if tagid1==tagid2:
                counter=counter+1
    return counter
tagcount2(1,2)

0

In [56]:
ratioleast = 0.75
countleast = 10
for tag1 in TagList:
    if tag1[2] == "hashtag" and tagcount1(tag1[0])>countleast:    
        for tag2 in TagList:
            if tag2[2] == "hashtag" and tagcount1(tag2[0]>countleast):
                if tag1 != tag2:
                    tag1id = tag1[0]
                    tag2id = tag2[0]
                    f1 = tagcount1(tag1id)
                    f2 = tagcount1(tag2id)
                    f3 = tagcount2(tag1id, tag2id)
                    if f3/f1 > ratioleast or f3/f2 >ratioleast:
                        if tag1id > tag2id:
                            temptagid = tag1id
                            tag1id = tag2id
                            tag2id = temptagid
                        if [tag1id, tag2id] not in SynonymsTagList:
                            SynonymsTagList.append([TagList[tag1id][1], TagList[tag2id][1]])

## Misspelling Tag

In [57]:
# Levenshtein distance calculation function
# Reference: https://rosettacode.org/wiki/Levenshtein_distance#Python
def ldistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1
    distances = range(len(s1) + 1)
    for index2, char2 in enumerate(s2):
        newDistances = [index2 + 1]
        for index1,char1 in enumerate(s1):
            if char1 == char2:
                newDistances.append(distances[index1])
            else:
                newDistances.append(1 + min((distances[index1],
                                             distances[index1 + 1],
                                             newDistances[-1])))
        distances = newDistances
    return distances[-1]

In [58]:
d1ratioleast = 0.30
d2ratioleast = 0.20
d3ratioleast = 0.10
for tag1 in TagList:
    if tag1[2] == "hashtag":    
        for tag2 in TagList:
            if tag2[2] == "hashtag":
                if tag1 != tag2:
                    tag1id=tag1[0]
                    tag2id=tag2[0]
                    f1=tagcount1(tag1id)
                    f2=tagcount1(tag2id)
                    distance=ldistance(tag1[1],tag2[1])
                    if distance<4:
                        if distance==1:
                            ratio=d1ratioleast
                        if distance==2:
                            ratio=d2ratioleast
                        if distance==3:
                            ratio=d3ratioleast
                        if f1/f2<ratio or f2/f1<ratio:
                            if f1>f2:
                                temptag =tag1id
                                tag1id=tag2id
                                tag2id=temptag
                                tempf=f1
                                f1=f2
                                f2=tempf
                            misspt0list=[]
                            for misspt in MisspellingsTagList:
                                misspt0list.append(misspt[0])
                            if not tag1id in misspt0list:
                                 MisspellingsTagList.append([tag1id, tag2id])
                            else:
                                missptori = []
                                for misspt in MisspellingsTagList:
                                    if misspt[0] == tag1id:
                                        missptori = misspt
                                        break
                                tagori1=""
                                tagori2=""
                                for tag in TagList:
                                    if tag[0]==missptori[0]:
                                        tagori1=tag[1]
                                    if tag[0]==missptori[1]:
                                        tagori2=tag[1]
                                f1ori=tagcount1(missptori[0])
                                f2ori=tagcount1(missptori[1])
                                rori=f1ori/f2ori
                                r=f1/f2
                                distanceori=ldistance(tagori1,tagori2)
                                if distance <distanceori:
                                    counter=-1
                                    for misspt in MisspellingsTagList:
                                        counter=counter+1
                                        if misspt[0]==tag1id:
                                            break
                                    MisspellingsTagList[counter]=[tag1id,tag2id]
                                else:
                                    if distance ==distanceori:
                                        if r<rori:
                                            counter=-1
                                            for misspt in MisspellingsTagList:
                                                counter=counter+1
                                                if misspt[0]==tag1id:
                                                    break
                                            MisspellingsTagList[counter]=[tag1id,tag2id]             

## TagCatagory

In [59]:
TagCatagoryList=[[1,"hashtag"], 
                 [2,"mention"],
                 [3,"domaintag"],
                 [4,"synonymstag"],
                 [5,"misspellingstag"]]

## TagCatagory_Tag

In [60]:
counter=0
for tag in TagList:
    if tag[2]=="hashtag":
        counter=counter+1
        TagCatagory_TagList.append([counter,1,tag[0]])
    if tag[2]=="mention":
        counter=counter+1
        TagCatagory_TagList.append([counter,2,tag[0]])
    domlist = []
    for dom in DomainTagList:
        domlist.append(dom[0])
    if tag[0] in domlist:
        counter=counter+1
        TagCatagory_TagList.append([counter,3,tag[0]])
    synlist=[]
    for syn in SynonymsTagList:
        synlist.append(syn[0])
    if tag[0] in synlist:
        counter=counter+1
        TagCatagory_TagList.append([counter,4,tag[0]])
    mislist=[]
    for mis in MisspellingsTagList:
        mislist.append(mis[0])
    if tag[0] in mislist:
        counter=counter+1
        TagCatagory_TagList.append([counter,5,tag[0]])

# Tagging dataframe

In [61]:
def list2tuplelist(list):
    tuplelist=[]
    for row in list:
        tuplelist.append(tuple(row))
    return tuplelist

In [62]:
labels=["tagID"]
DomainTag=pd.DataFrame.from_records(list2tuplelist(DomainTagList),columns=labels)

In [63]:
DomainTag.head()

,tagID
0,2
1,16
2,29
3,61
4,90


In [64]:
labels=["tagID1","tagID2"]
SynonymsTag=pd.DataFrame.from_records(list2tuplelist(SynonymsTagList),columns=labels)

In [65]:
SynonymsTag.head()

,tagID1,tagID2


In [66]:
labels=["tagID1","tagID2"]
MisspellingsTag=pd.DataFrame.from_records(list2tuplelist(MisspellingsTagList),columns=labels)

In [67]:
MisspellingsTag.head()

,tagID1,tagID2


In [68]:
labels=["trtID","TagCatagorycatagoryID","TagTagID"]
TagCatagory_Tag=pd.DataFrame.from_records(list2tuplelist(TagCatagory_TagList),columns=labels)

In [69]:
TagCatagory_Tag.head()

,trtID,TagCatagorycatagoryID,TagTagID
0,1,2,1
1,2,1,2
2,3,3,2
3,4,2,3
4,5,2,4


In [70]:
labels=["catagoryID","catagoryName"]
TagCatagory=pd.DataFrame.from_records(list2tuplelist(TagCatagoryList),columns=labels)

In [71]:
TagCatagory.head()

,catagoryID,catagoryName
0,1,hashtag
1,2,mention
2,3,domaintag
3,4,synonymstag
4,5,misspellingstag


# export to csv

In [72]:
#Player = pd.DataFrame(columns = ["personID"])
Player.to_csv(r'Player.csv',index=False, encoding="utf-8-sig")
#Player_Fan = pd.DataFrame(columns = ["prfID", "PlayerpersonID", "FanpersonID"])
Player_Fan.to_csv(r'Player_Fan.csv',index=False, encoding="utf-8-sig")
#Fan = pd.DataFrame(columns = ["personID"])
Fan.to_csv(r'Fan.csv',index=False, encoding="utf-8-sig")
#Fan_Team = pd.DataFrame(columns = ["frtID", "FanpersonID", "TeamteamID"])
Fan_Team.to_csv(r'Fan_Team.csv',index=False, encoding="utf-8-sig")
#Person = pd.DataFrame(columns = ["personID", "personName", "locationID"])
Person.to_csv(r'Person.csv',index=False, encoding="utf-8-sig")
#Team = pd.DataFrame(columns = ["teamID", "teamName", "locationID"])
Team.to_csv(r'Team.csv',index=False, encoding="utf-8-sig")
#Person_SNSUser = pd.DataFrame(columns = ["prsID", "PersonpersonID", "SNSUserSNSUserID"])
Person_SNSUser.to_csv(r'Person_SNSUser.csv',index=False, encoding="utf-8-sig")
#Location = pd.DataFrame(columns = ["locationID", "locationName"])
Location.to_csv(r'Location.csv',index=False, encoding="utf-8-sig")
#Team_SNSUser = pd.DataFrame(columns = ["trsID", "TeamteamID", "SNSUserSNSUserID"])
Team_SNSUser.to_csv(r'Team_SNSUser.csv',index=False, encoding="utf-8-sig")
#SNSUser = pd.DataFrame(columns = ["SNSUserID", "SNSType", "accountID", "userName", "followersCount", "postsCount"])
SNSUser.to_csv(r'SNSUser.csv',index=False, encoding="utf-8-sig")
#Post = pd.DataFrame(columns = ["SNSpostID", "postID", "SNSUserID", "content", "favoriteCount", "createdTime"])
Post.to_csv(r'Post.csv',index=False, encoding="utf-8-sig")
#Post_Tag = pd.DataFrame(columns = ["prtID", "PostSNSpostID", "TagTagID"])
Post_Tag.to_csv(r'Post_Tag.csv',index=False, encoding="utf-8-sig")
#Tag = pd.DataFrame(columns = ["tagID", "tagText", "tagType"])
Tag.to_csv(r'Tag.csv',index=False, encoding="utf-8-sig")
#DomainTag = pd.DataFrame(columns = ["tagID"])
DomainTag.to_csv(r'DomainTag.csv',index=False, encoding="utf-8-sig")
#SynonymsTag = pd.DataFrame(columns = ["tagID1", "tagID2"])
SynonymsTag.to_csv(r'SynonymsTag.csv',index=False, encoding="utf-8-sig")
#MisspellingsTag = pd.DataFrame(columns = ["tagID1", "tagID2"])
MisspellingsTag.to_csv(r'MisspellingsTag.csv',index=False, encoding="utf-8-sig")
#TagCatagory_Tag = pd.DataFrame(columns = ["trtID", "TagCatagorycatagoryID", "TagTagID"])
TagCatagory_Tag.to_csv(r'TagCatagory_Tag.csv',index=False, encoding="utf-8-sig")
#TagCatagory = pd.DataFrame(columns = ["catagoryID", "catagoryName"])
TagCatagory.to_csv(r'TagCatagory.csv',index=False, encoding="utf-8-sig")